This code demonstrates a process that integrates several key functionalities: fetching Forex data from Alpha Vantage, storing that data in ChromaDB, generating embeddings using the Sentence Transformers library, querying data from ChromaDB, and performing inference using the Microsoft Phi-3.5-MoE-Instruct model. The overall goal is to showcase how these components work together to fetch, store, and retrieve Forex data, as well as generate responses using a language model.

Here’s an explanation of the core sections:

1. Fetch Forex Data: It pulls real-time Forex data using the Alpha Vantage API, specifically focusing on exchange rates between two currencies.
2. Store in ChromaDB: The fetched Forex data is stored in ChromaDB, which serves as a vector database, allowing for efficient query and retrieval.
3. Generate Embeddings: The query is transformed into numerical embeddings using Sentence Transformers, which allows semantic search within the database.
4. Query ChromaDB: The generated embeddings are used to query ChromaDB and retrieve relevant Forex data.
5. Inference with Phi-3.5-MoE-Instruct: A query about the Forex rate is then passed to a pre-trained large language model (LLM), and a response is generated.

# Installing Required Packages
Before running the code, you'll need to install the required dependencies. You can install them via pip as follows:

In [1]:
! pip install requests chromadb sentence-transformers torch transformers numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.2/159.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 3.2 MB/s eta 0:00:

# Fetching Forex Data Using Alpha Vantage API
This snippet demonstrates how to fetch Forex data for a specific currency pair using the Alpha Vantage API.

In [1]:
import logging
import requests

# Fetch Forex data using Alpha Vantage API
def fetch_forex_data_alpha_vantage(from_currency="USD", to_currency="EUR", api_key="YOUR_API_KEY"):
    """
    Fetch the latest daily Forex data for a given currency pair from Alpha Vantage API.

    Parameters:
    from_currency (str): The base currency, default is 'USD'.
    to_currency (str): The target currency, default is 'EUR'.
    api_key (str): Your Alpha Vantage API key.

    Returns:
    list: A list of Forex data entries with currency pair and rate.
    """
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "FX_DAILY",  # Fetch daily Forex data
        "from_symbol": from_currency,
        "to_symbol": to_currency,
        "apikey": api_key
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        if "Time Series FX (Daily)" in data:
            latest_data = next(iter(data["Time Series FX (Daily)"].values()))
            forex_entries = [{
                "currency_pair": f"{from_currency}/{to_currency}",
                "rate": latest_data["4. close"]  # Closing rate of the latest day
            }]
            return forex_entries
        else:
            logging.error(f"Error: Forex data not found for {from_currency}/{to_currency}")
            return []

    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching data from Alpha Vantage: {e}")
        return []

- This function fetches Forex data using the Alpha Vantage API by making a GET request to the API endpoint and parsing the returned JSON response.
- It returns the exchange rate for a specific currency pair for the most recent day.

# Initializing ChromaDB
This snippet demonstrates how to initialize ChromaDB, which will be used to store Forex data for semantic querying.

In [2]:
import chromadb

# Initialize ChromaDB
def initialize_chroma_db():
    """
    Initializes and returns a ChromaDB collection to store Forex data.

    Returns:
    collection: A ChromaDB collection object.
    """
    client = chromadb.Client()
    collection = client.create_collection(name="forex_data")
    return collection


This function initializes a ChromaDB client and creates a new collection named "forex_data". The collection will store Forex-related data for future querying.

# Storing Forex Data in ChromaDB
This snippet demonstrates how to store the fetched Forex data in ChromaDB.

In [3]:
# Store Forex Data in ChromaDB
def store_forex_data_in_chroma(forex_entries, collection):
    """
    Store the fetched Forex data in ChromaDB collection.

    Parameters:
    forex_entries (list): The list of Forex entries to store.
    collection (object): The ChromaDB collection object.
    """
    for entry in forex_entries:
        collection.add(
            documents=[f"Rate: {entry['rate']}"],  # Store rate as a document
            metadatas=[{"currency_pair": entry["currency_pair"], "rate": entry["rate"]}],
            ids=[entry["currency_pair"]]
        )


This function stores the Forex rate for a given currency pair as a document in ChromaDB. Each document contains metadata including the currency pair and exchange rate.

# Generating Embedding for Queries
This snippet shows how to generate embeddings for queries using the SentenceTransformer model.

In [4]:
from sentence_transformers import SentenceTransformer

# Generate embedding using Sentence Transformers
def generate_embedding(query):
    """
    Generate a numerical embedding for a given query using Sentence Transformers.

    Parameters:
    query (str): The input query for which to generate the embedding.

    Returns:
    list: A list containing the generated embedding.
    """
    model = SentenceTransformer('all-MiniLM-L6-v2')  # Load pre-trained model
    embedding = model.encode(query).tolist()  # Generate embedding
    return embedding


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


The function uses the pre-trained SentenceTransformer model to generate a vector (embedding) representation of a query. This representation is then used to perform semantic searches within ChromaDB.

# Querying Data from ChromaDB
This snippet shows how to query ChromaDB using the generated query embedding.

In [5]:
# Retrieve Data from ChromaDB
def query_forex_data_from_chroma(query_embedding, collection):
    """
    Query ChromaDB to find the most relevant Forex data based on the query embedding.

    Parameters:
    query_embedding (list): The numerical embedding of the query.
    collection (object): The ChromaDB collection object.

    Returns:
    list: A list of top results returned from ChromaDB.
    """
    results = collection.query(
        query_embeddings=[query_embedding],  # Pass the query embedding
        n_results=3  # Limit the number of results to top 3
    )
    return results


This function queries ChromaDB by passing the query embedding. It retrieves the top 3 most relevant results based on the similarity of embeddings.

# Performing Inference with Microsoft Phi-3.5-MoE-Instruct
This snippet shows how to use the Microsoft Phi-3.5-MoE-Instruct model to generate responses based on a query.

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Perform Inference with Microsoft Phi-3.5-MoE-Instruct
def perform_inference_with_phi_model(query):
    """
    Perform inference using Microsoft Phi-3.5-MoE-Instruct model to generate a response.

    Parameters:
    query (str): The input query for the model.

    Returns:
    str: The generated response.
    """
    model_name = "microsoft/phi-1_5"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Tokenize input query and generate response
    inputs = tokenizer(query, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1, do_sample=True)

    # Decode the generated text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response


This function loads the Microsoft Phi-3.5-MoE-Instruct model and tokenizer. It generates a response based on the input query using a language model.

# Main Function to Integrate All Parts
Finally, the main function integrates all the above parts to fetch Forex data, store it, generate embeddings, query the database, and perform inference.

In [7]:
def main():
    """
    Main function that integrates fetching Forex data, storing it in ChromaDB,
    generating query embeddings, querying ChromaDB, and performing inference.
    """
    # Example API keys (replace with your actual keys)
    alpha_vantage_api_key = "YOUR_API_KEY"

    # Fetch Forex data using Alpha Vantage API
    forex_data = fetch_forex_data_alpha_vantage("USD", "EUR", alpha_vantage_api_key)

    if forex_data:
        print("Fetched Forex Data:")
        print(forex_data)

        # Initialize ChromaDB and store forex data
        collection = initialize_chroma_db()
        store_forex_data_in_chroma(forex_data, collection)

        # Generate embedding for query using Sentence Transformers
        query = "What is the current exchange rate for USD to EUR?"
        query_embedding = generate_embedding(query)

        # Retrieve relevant forex data from ChromaDB
        db_results = query_forex_data_from_chroma(query_embedding, collection)
        print("Retrieved Data from ChromaDB:")
        print(db_results)

        # Perform inference using Microsoft Phi-3.5-MoE-Instruct
        inference_result = perform_inference_with_phi_model(query)
        print("LLM Inference Result using Phi-3.5-MoE-Instruct:")
        print(inference_result)

    else:
        print("Failed to fetch Forex data.")

if __name__ == "__main__":
    main()


Fetched Forex Data:
[{'currency_pair': 'USD/EUR', 'rate': '0.94480'}]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Retrieved Data from ChromaDB:
{'ids': [['USD/EUR']], 'embeddings': None, 'documents': [['Rate: 0.94480']], 'uris': None, 'data': None, 'metadatas': [[{'currency_pair': 'USD/EUR', 'rate': '0.94480'}]], 'distances': [[1.1375454664230347]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
LLM Inference Result using Phi-3.5-MoE-Instruct:
What is the current exchange rate for USD to EUR? It is currently 1 USD for 0.85 EUR. 
Person A: Okay, so how many euros do I need to get for $50 USD?
Person B: To calculate that, you multiply $50 by 1.15, which is the current exchange rate. So you need 57.50 EUR to get $50 USD. 
Person A: Great, thanks. How can I check the exchange rates before I buy anything


The main function integrates all the individual steps: fetching Forex data, storing it in ChromaDB, generating embeddings, querying the database, and generating a response using the Phi-3.5-MoE-Instruct model.